# GAN 직접 만들어보기!(Feat. 쉽게 씌여진 GAN)

* This Notebook referenced <code><a href='https://dreamgonfly.github.io/2018/03/17/gan-explained.html'>쉽게 씌여진 GAN</a></code>

---

* 오늘 풀어볼 문제(GAN Tutorial)
    * 0부터 9까지 숫자 모양의 손글씨 이미지를 생성하는 문제
    * Dataset: MNIST(손글씨 데이터셋으로 딥러닝계의 "Hello World!"라고 합니다!)
    * 비교적 간단한 문제지만 GAN의 원리는 어떤 문제에도 동일하게 적용되기 때문에 유용한 예제가 될 것이라고 합니다!

* 전체코드는 <a href='https://github.com/dreamgonfly/GAN-tutorial'>여기서</a> 확인 가능하다고 합니다!(쉽씌 GAN 저자 조용래님의 코드)

## 시작하기

## Library Import

가장 먼저 할 일은 library를 불러오는 일입니다.<br>
본 튜토리얼에서 사용하는 MNIST Dataset은 Pytorch torchivision에서 받을 수 있습니다.

In [2]:
# <코드1> 라이브러리 및 데이터 불러오기

import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

## 데이터 전처리

### transform
* 전처리 방식 지정

<code>torchvision.transforms</code>?

In [6]:
'''
transform = transforms.Compose([
    transforms.ToTensor(), # 데이터를 파이토치의 Tensor 형식으로 바꿔준다.
    transforms.Normalize(mean=(0.5,), std=(0.5,))  # 픽셀 값 0~1 -> -1 ~ 1 변경
])
'''

transform = transforms.Compose([
    transforms.ToTensor(),                          # 데이터를 파이토치의 Tensor 형태로 변환
    transforms.Normalize(mean=(0.5,), std=(0.5,))  # 0~1인 픽셀값 -> -1 ~ 1로 변경! 
])


### Load Data
* 데이터 불러오기 

In [7]:
'''
# Original Code

mnist = datasets.MNIST(root='data', download=True, transform=transform)
'''

# Review
# MNIST 데이터셋을 불러옵니다. 지정한 폴더에 없을 경우 자동으로 다운로드 합니다.

mnist = datasets.MNIST(root='data', download=True, transform=transform)

### DataLoader
* 데이터를 한 번에 batch_size만큼만 가져오는 dataloader를 만든다.

In [8]:
from torch.utils.data import DataLoader

'''
# Original Code

dataloader = DataLoader(mnist, batch_size=60, shuffle=True)
'''

# Review
dataloader = DataLoader(mnist, batch_size=60, shuffle=True)  # batch_size:
                                                             # shuffle=True: have the data reshuffled at every epoch (default: ``False``).
                                                             # https://pytorch.org/docs/1.1.0/_modules/torch/utils/data/dataloader.html

In [9]:
'''
===========
  Summary
===========

# <코드1> 라이브러리 및 데이터 불러오기

import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

#데이터 전처리 방식을 지정한다.
transform = transforms.Compose([
  transforms.ToTensor(), # 데이터를 파이토치의 Tensor 형식으로바꾼다.
  transforms.Normalize(mean=(0.5,), std=(0.5,)) # 픽셀값 0 ~ 1 -> -1 ~ 1
])

#MNIST 데이터셋을 불러온다. 지정한 폴더에 없을 경우 자동으로 다운로드한다.
mnist =datasets.MNIST(root='data', download=True, transform=transform)

#데이터를 한번에 batch_size만큼만 가져오는 dataloader를 만든다.
dataloader =DataLoader(mnist, batch_size=60, shuffle=True)
'''

"\n=======\nSummary\n=======\n\n# <코드1> 라이브러리 및 데이터 불러오기\n\nimport torch\nimport torch.nn as nn\nfrom torch.optim import Adam\nfrom torchvision import datasets, transforms\nfrom torch.utils.data import DataLoader\nfrom torch.autograd import Variable\n\n#데이터 전처리 방식을 지정한다.\ntransform = transforms.Compose([\n  transforms.ToTensor(), # 데이터를 파이토치의 Tensor 형식으로바꾼다.\n  transforms.Normalize(mean=(0.5,), std=(0.5,)) # 픽셀값 0 ~ 1 -> -1 ~ 1\n])\n\n#MNIST 데이터셋을 불러온다. 지정한 폴더에 없을 경우 자동으로 다운로드한다.\nmnist =datasets.MNIST(root='data', download=True, transform=transform)\n\n#데이터를 한번에 batch_size만큼만 가져오는 dataloader를 만든다.\ndataloader =DataLoader(mnist, batch_size=60, shuffle=True)\n"

라이브러리와 데이터를 불러왔다. 이제 다음으로 할 일은 GAN의 2가지 요소인 생성자와 구분자를 만들어보는 일이다. 우선 생성자(Generator)를 먼저 만들어보자.

## Generator

* 생성자(Generator)
    * 랜덤 벡터 'z'를 입력으로 받아 가짜 이미지를 출력하는 함수이다.
    * 여기서 'z'는 단순하게 균등 분포(Uniform Distribution)나 정규 분포(Normal Distribution)에서 무작위로 추출된 값이다.
    * 생성자는 이렇게 단순한 분포를 사람 얼굴 이미지와 같은 복잡한 분포로 매핑(Mapping)하는 함수라고 볼수 있다.
    * 생성자 모델에 충분한 수의 매개 변수가 있다면 어떤 복잡한 분포도 근사할 수 있다고 알려져있다.


* 정리
    * Generator는 Uniform Distribution 또는 Noraml Distribution에서 무작위로 추출한 z로부터 가짜 이미지를 생성한다.<br>
    * Generator는 이처럼 단순한 분포를 사람 얼굴 이미지와 같은 복잡한 분포로 Mapping시켜주는 함수이다.
    * 일반적으로 Generator Model에 충분한 수의 매개 변수가 있다면 어떤 복잡한 분포도 근사할 수 있다고 알려져있다.

*신기하다... 단순 분포의 값을 사람 얼굴 이미지와 같은 복잡한 분포로 어떻게 Mapping시켜주는걸까?*<br>
Tutorial이 끝나갈때 쯤에는 알 수 있지 않을까? 기대되는 부분이다!

* 생성자 모델에 충분한 수의 매개 변수가 있다면 어떤 복잡한 분포도 근사할 수 있다.

<img src="./img/gan_distribution_mapping.png"></img>

* 'z'벡터가 존재하는 공간 == Latent Space
    * 'z'벡터가 존재하는 공간을 <code>잠재공간(Latent Space)</code>라고도 부르다.
    * 본 Tutorial에서는 Latent Space의 크기를 임의로 100차원으로 뒀다.
    * 제한은 없으나 우리가 나타내려고 하는 대상의 정보를 충분히 담을 수 있을 만큼 커야한다.
    * GAN은 우리가 이해할 수 없는 방식이지만 'z'벡터의 값을 이미지의 속성에 매핑시키기 떄문이다. <- What?!
    * 뒤에서 살펴볼 GAN의 파생 모델에서 잠재 공간의 의미를 더욱 자세히 이해할 수 있다.

* 생성자에 충분한 수의 매개 변수를 확보하기 위해, 본 구현에서는 선형 레이어를 쌓아 생성자를 만들었다.
    * 선형 레이어(Linear Layer, Fully Connected Layer, Linear Transformation)는 속해져있는 모든 뉴런이 이전 레이어의 모든 뉴런과 연결되는 가장 단순한 구조의 레이어이다. (Fully connected Layer를 의미하는게 아닌가요?)
    * 이 모델에서는 100차원의 랜덤 벡터를 받아 이를 256개의 뉴런을 가진 레이어로 보내고 다시 레이어의 크기를 512, 1024로 점점 증가시켰다.
    * 마지막에는 출력을 MNIST 이미지의 크기로 맞추기 위해 레이어의 크기를 28x28로 줄였다.
    
* 정리
    * Generator에 충분한 수의 매개 변수 확보를 위한 선형 레이어 쌓기
    * Fully connected Layer를 구성함
    * <code>100차원의 Random Vector Z -> 256 -> 512 -> 1024 -> 28*28 변환(MNIST 이미지의 크기로 맞추기 위함)</code>

* 활성화 함수로는 Leaky ReLU를 사용했다.
    * Leaky ReLU에 대한 설명은 생략
* 여러 레이어와 활성 함수를 쌓은 덕분에 MNIST 데이터 분포를 근사할 수 있는 충분한 표현력(Representation Power)를 얻을 수 있었다.
* 더욱 복잡한 문제를 풀기 위해서는 더 깊은 레이어 구조와 더 많은 양의 매개 변수가 필요할 것이다.

### Generator Summary

* Generator는 Uniform Distribution 또는 Noraml Distribution에서 <code>무작위로 추출한 z로부터 가짜 이미지를 생성</code>한다.
* Generator는 이처럼 단순한 분포를 사람 얼굴 이미지와 같은 복잡한 분포로 Mapping시켜주는 함수이다.
* 일반적으로 Generator Model에 충분한 수의 매개 변수가 있다면 어떤 복잡한 분포도 근사할 수 있다고 알려져있다.
* Generator에 충분한 수의 매개 변수(Parameter)를 확보하기 위해서는 선형 레이어를 쌓아햐 한다.
* Fully Connected Layer로 구성을해준다.
* 차원수: <code>100차원의 Random Vector Z -> 256 -> 512 -> 1024 -> 28*28 변환(MNIST 이미지의 크기로 맞추기 위함)</code>
* 여러 레이어와 Leaky ReLU(활성화 함수를 쌓아 충분한 표현력(Representation Power)를 얻는다.
* 더 복잡한 문제를 풀기 위해서는 더 깊은 레이어 구조와 더 많은 양의 매개 변수가 필요하다.

In [10]:
# <코드 2> GAN의 생성자(Generator)

# 생성자는 랜덤 벡터 z를 입력으로 받아 가짜 이미지를 출력한다.

class Generator(nn.Module):
    
    # 네트워크 구조
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=100, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=256, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=512, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=1024, out_features=28*28),
            nn.Tanh())
    
    # (batch_size x 100) 크기의 랜덤 벡터를 받아
    # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
    # batch_size는 60이기에 입력 값의 형상은 (60, 100)
    # (60, 100) -> (60, 256), (60, 512), (60, 1024), (60, 28*28) -> (-1, 1, 28, 28)
    
    def forward(self, inputs):
        return self.main(inputs).view(-1, 1, 28, 28)

##  Discriminator

* 구분자(Discriminator)
    * Discriminator는 이미지를 입력으로 받고 그 이미지가 진짜일 확률을 0과 1 사이의 숫자 하나로 출력하는 함수다.
    * 구분자의 구현은 생성자와 마찬가지로 4개의 선형 레이어를 쌓았다.
    * 마찬가지 활성화 함수로는 LeakyReLU를 사용했다.
    * 입력 이미지의 크기인 28x28이 1024, 512, 256으로 줄다 마지막에는 확률값을 나타내는 숫자 하나를 출력한다.

In [12]:
# <코드 3> GAN의 구분자 (Discriminator)

# 구분자는 이미지를 입력으로 받아 이미지가 진짜인지 가짜인지를 출력한다.
'''
class Discriminator(nn.Module):
    
    # 네트워크 구조
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=1024, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=512, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid())
        
    # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
    # 이미지가 진짜일 확률을 0~1 사이로 출력
    def forward(self, inputs):
        inputs = inputs.view(-1, 28* 28)
        return self.main(inputs)
'''    
    
    
class Discriminator(nn.Module):
    
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=1024, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=512, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid())
                
    # (batch_size x 1 x 28 x 28)크기의 이미지를 입력으로 받아
    # 이미지가 진짜일 확률을 0~1 사이의 값으로 출력한다
    def forward(self, inputs):
        inputs = inputs.view(-1, 28*28)
        return self.main(inputs)
    
    
    # (batch_size, 1, 28, 28) -> (batch_size, 28, 28) -> (batch_size, 1024) -> (batch_size, 512) ->
    # -> (batch_size, 256) -> (batch_size, 1)

## Generator와 Discriminator 만들기

In [13]:
G = Generator()
D = Discriminator()

* 학습시키기
    * 이제부터는 만들어진 네트워크 구조를 학습하는 방법에 대해 알아보자
    * 학습을 위해서는 모델을 평가할 수 있어야 한다.
        * 모델의 평가 지표가 좋아지는 방향으로 매개 변수를 업데이트 할 것이기 떄문이다.
    * Discriminator(구분자)의 출력값은 이미지가 진짜일 확률이고, 이 확률이 얼마나 정답과 가까운지를 측정하기 위해 Binary Cross Entropy Loss Function을 사용할 것이다.
    * 이 함수는 구분자가 출력한 확률값이 정답에 가까우면 낮아지고 구분자가 출력한 값이 정답에서 멀어지면 높아진다
    * 즉 학습이 잘 됬을수록 손실 함수의 값은 작아진다!

* 매개변수 최적화 함수
    * Generator(생성자)와 Discriminator(구분자)의 매개 변수를 업데이트하는 최적화 함수가 각각 하나 필요하다
    * 가장 널리 사용되는 Adam을 사용하도록 한다.
    * Adam은 매개변수마다 업데이트 속도를 최적으로 조절하는 효율적인 최적화 기법이다

In [14]:
# <코드 5> 손실 함수와 최적화 기법 지정하기

# Binary Cross Entropy loss
criterion = nn.BCELoss()  ## Binary Cross Entropy Loss

# 생성자의 매개 변수를 최적화하는 Adam optimizer
G_optimizer = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))  # what is betas??
# 구분자의 매개 변수를 최적화하는 Adam optimizer 
D_optimizer = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))  #  coefficients 
                                                                   # used for computing running averages of gradient and its square (default: (0.9, 0.999))


* Epoch
    * 모델 학습을 위해 전체 데이터셋을 여러 번 돌며 매개 변수를 조금씩 업데이트한다.
    * 데이터셋을 한 번 도는 것을 1 Epoch이라고 한다.
    * 여기서는 100 Epoch을 돌 것이다.
    * 각 Epoch마다 Batch Size인 60만큼 데이터를 가져와 모델을 학습시킨다.
    * MNIST 학습 데이터의 개수가 6만개이니 1에폭마다 1000번씩 학습이 이루어지는 셈이다!
        * e.g) 1-Epoch -> 60개의 batch가 1000번 학습된다(데이터의 개수가 6만개 이기 떄문)

In [21]:
# <코드 6> 모델 학습을 위한 반복문


# 데이터셋을 100번 돌며 학습한다
for epoch in range(100):
    
    # 한 번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
        
    # 데이터를 파이토치의 변수로 변환한다.
        real_data = Variable(real_data)
        
        # ...  코드 7과 이어지는구나!


## Training Discriminator

* 먼저 Discriminator를 학습시킨다
    * Discriminator는 진짜 이미지를 입력하면 1에 가까운 확률값을 출력한다
    * 가짜 데이터를 입력하면 0에 가까운 확률값을 출력한다.
    * 따라서 Discriminator의 손실 함수는 두 가지의 합으로 이루어진다.
    * (진짜 이미지를 입력했을 떄의 출력값과 1의 차이) + (가짜 이미지를 입력했을 때 출력값과 0의 차이)
    * 이 손실 함수의 값을 최소화 하는 방향으로 매개변수가 업데이트 된다.
    
    
* 파이토치에서는 역전파를 통해 간단한 방법으로 계산된 각 변수의 미분 값을 구할 수 있다.
    * 그 상태에서 최적화 함수를 실행시키면 매개 변수가 한 번 업데이트된다 

In [ ]:
# <코드7> Training Discriminator
    ## 정답지에 해당하는 변수를 만든다
    # Image가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다
    target_real = Variable(torch.ones(batch_size, 1))
    target_fake = Variable(torch.zeros(batch_size, 1))
    
    ## Train real_data
    # 진짜 이미지를 구분자에 넣는다
    D_result_from_real = D(real_data)
    # 구분자의 출력값이 정답지인 1에서 멀수록 loss가 높아진다
    D_loss_real = criterion(D_result_from_real, target_real)
   
    ## Generating fake image 
    # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
    z = Variable(torch.randn((batch_size, 100)))
    # 생성자로 가짜 이미지를 생성한다.
    fake_data = G(z)  # fake_data == 가짜 이미지 데이터, 
                      # 이 데이터를 Discriminator로 학습시켜 loss를 구한다!
                      # 가짜 이미지를 잘 구분할수록 loss는 줄어들고 반대의 경우 loss는 커지겠지!
            
    # 생성자가 만든 가짜 이미지를 구분자에 넣는다
    D_result_from_fake = D(fake_data)
    
    # 구분자의 출력값이 정답인 0에서 멀수록 loss가 높아진다
    D_loss_fake = criterion(D_result_from_fake, target_fake)
    
    ## Multi-Loss
    # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
    D_loss = D_loss_real + D_loss_fake
    
    # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
    D.zero_grad()
    
    # 역전파를 통해 매개변수의 loss에 대한 미분값을 계산한다.
    D_loss.backward()
    
    # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트하낟.
    D_optimizer.step()


와...이거 좋은데~? -20.06.01.mon-

In [22]:
# <code 8> 생성자 학습시키기
# 잠깐, 위에서 생성자 학습시키거 아녔어?? Fake Image Data 생성했는데?!
# 일단 진행해보자.

    # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
    z = Variable(torch.randn((batch_size, 100)))
    z = z.cuda()
    
    # 생성자로 가짜 이미지를 생성한다.
    fake_data = G(z)
    
    # 생성자가 만든 가짜 이미지를 구분자에 넣는다
    D_result_from_fake = D(fake_data)
    # 생성자의 입장에서 구분자의 출력값이 1에서 멀어질수록 loss가 높아진다.
    # 위조지폐범의 입장에서 경찰이 진짜 지폐로 분류하기 원할테니깐!(1은 진짜지폐로 분류!)
    G_loss = criterion(D_result_from_fake, target_real)
    
    # 생성자의 매개 변수의 미분값을 0로 초기화
    G.zero_grad()
    
    # 역전파를 통해 매개 변수의 loss에 대한 미분값 계산
    G_loss.backward()
    
    # 최적화 기법을 이용해 생성자의 매개 변수 업데이트
    G_optimizer.step()
    
    
# 위에서는 Discriminator의 loss를 학습하기 위해 Generator로 Fake Image를 만들어 학습했다면
# <Code 8>은 Generator를 학습시키기 위함이군!

IndentationError: unexpected indent (<ipython-input-22-8aa9788c61e5>, line 6)

## Integration

<code>Code 6, 7, 8</code>을 합쳐준다!

In [13]:
# <코드 6> 모델 학습을 위한 반복문

# 데이터셋을 100번 돌며 학습한다
for epoch in range(100):
    
    # 한 번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
        
    # 데이터를 파이토치의 변수로 변환한다.
        real_data = Variable(real_data)
        
        # ...  코드 7과 이어지는구나!
            # <코드7> Training Discriminator
        ## 정답지에 해당하는 변수를 만든다
        # Image가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다
        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))

        ## Train real_data
        # 진짜 이미지를 구분자에 넣는다
        D_result_from_real = D(real_data)
        # 구분자의 출력값이 정답지인 1에서 멀수록 loss가 높아진다
        D_loss_real = criterion(D_result_from_real, target_real)

        ## Generating fake image 
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)  # fake_data == 가짜 이미지 데이터, 
                          # 이 데이터를 Discriminator로 학습시켜 loss를 구한다!
                          # 가짜 이미지를 잘 구분할수록 loss는 줄어들고 반대의 경우 loss는 커지겠지!

        # 생성자가 만든 가짜 이미지를 구분자에 넣는다
        D_result_from_fake = D(fake_data)

        # 구분자의 출력값이 정답인 0에서 멀수록 loss가 높아진다
        D_loss_fake = criterion(D_result_from_fake, target_fake)

        ## Multi-Loss
        # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
        D_loss = D_loss_real + D_loss_fake

        # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
        D.zero_grad()

        # 역전파를 통해 매개변수의 loss에 대한 미분값을 계산한다.
        D_loss.backward()

        # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트하낟.
        D_optimizer.step()
        
# <code 8> 생성자 학습시키기
# 잠깐, 위에서 생성자 학습시키거 아녔어?? Fake Image Data 생성했는데?!
# 일단 진행해보자.

        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
#         z = z.cuda()

        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)

        # 생성자가 만든 가짜 이미지를 구분자에 넣는다
        D_result_from_fake = D(fake_data)
        # 생성자의 입장에서 구분자의 출력값이 1에서 멀어질수록 loss가 높아진다.
        # 위조지폐범의 입장에서 경찰이 진짜 지폐로 분류하기 원할테니깐!(1은 진짜지폐로 분류!)
        G_loss = criterion(D_result_from_fake, target_real)

        # 생성자의 매개 변수의 미분값을 0로 초기화
        G.zero_grad()

        # 역전파를 통해 매개 변수의 loss에 대한 미분값 계산
        G_loss.backward()

        # 최적화 기법을 이용해 생성자의 매개 변수 업데이트
        G_optimizer.step()

    print(epoch, "update D_Loss:", D_loss)
    print(epoch, "update G_Loss:", G_loss)

# 위에서는 Discriminator의 loss를 학습하기 위해 Generator로 Fake Image를 만들어 학습했다면
# <Code 8>은 Generator를 학습시키기 위함이군!

0 update D_Loss: tensor(0.2865, grad_fn=<AddBackward0>)
0 update G_Loss: tensor(1.5037, grad_fn=<BinaryCrossEntropyBackward>)
1 update D_Loss: tensor(0.2832, grad_fn=<AddBackward0>)
1 update G_Loss: tensor(2.8324, grad_fn=<BinaryCrossEntropyBackward>)
2 update D_Loss: tensor(0.5949, grad_fn=<AddBackward0>)
2 update G_Loss: tensor(2.4875, grad_fn=<BinaryCrossEntropyBackward>)
3 update D_Loss: tensor(0.7003, grad_fn=<AddBackward0>)
3 update G_Loss: tensor(3.1609, grad_fn=<BinaryCrossEntropyBackward>)


KeyboardInterrupt: 

## Run on CUDA GPU

In [28]:
# CUDA 사용환경 check!! 

torch.cuda.is_available()

True

### PyTorch GPU 사용법

* PyTorch - CUDA(GPU)사용하기
    * Reference - <a href='https://yonghyuc.wordpress.com/2019/08/06/pytorch-cuda-gpu-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0/'>PyTorch CUDA(GPU 사용하기) 조용현님의 블로그</a>

* Machine을 학습시킬 때 GPU를 사용하게 되면 월등히 성능이 좋아진다.
    * 먼저 지금 GPU를 사용하여 학습을 하는지 CPU를 사용하고 있는지 확인부터해보자

* Reference:
    * <a href='https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html'>PyTorch Example - Tensors</a>
    * Numpy와 달리 PyTorch Tensor는 GPU를 활용하여 수치 연산을 가속화 할 수 있다.
    * GPU에서 PyTorch Tensor를 실행하기 위해서는 단지 새로운 자료형으로 변환해주기만 하면 된다(cast)

In [33]:
import torch

# Returns a bool indicating if CUDA is currently available
torch.cuda.is_available()

# Return the index of a currently selected device
torch.cuda.current_device()

0

In [34]:
# <코드 6> 모델 학습을 위한 반복문

# Tensor를 새로운 자료형으로 변환하기(Cast)
# dtype, device 정의
'''
# Use This
dtype = torch.float
device = torch.device("cpu")
device = torch.device("cuda:0")  # GPU 실행방법

# But it doesn't work in Variable

'''

'''
# When Using Variable
https://discuss.pytorch.org/t/will-all-variables-created-in-network-run-in-gpu/9968/7

If you create a Variable in the forward(), it cannot switch to cuda automatically. 
you must specify .cuda() . Something like:

h = Variable(torch.randn(2, batch_size, 256*2))
h = h.cuda()

My English is poor. Hope it can help you.

'''

device = torch.device("cuda:0")

G = Generator().cuda()
D = Discriminator().cuda()

# 데이터셋을 100번 돌며 학습한다
for epoch in range(100):
    # 한 번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
        
    # 데이터를 파이토치의 변수로 변환한다.
        real_data = Variable(real_data)
        
# <코드7> Training Discriminator
        ## 정답지에 해당하는 변수를 만든다
        # Image가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다
        target_real = Variable(torch.ones(batch_size, 1))
        target_real = target_real.cuda()
        target_fake = Variable(torch.zeros(batch_size, 1))
        target_fake = target_fake.cuda()

        ## Train real_data
        # 진짜 이미지를 구분자에 넣는다
        D_result_from_real = D(real_data).cuda()
        # 구분자의 출력값이 정답지인 1에서 멀수록 loss가 높아진다
        D_loss_real = criterion(D_result_from_real, target_real)

        ## Generating fake image 
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        z = z.cuda()
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)  # fake_data == 가짜 이미지 데이터, 
                          # 이 데이터를 Discriminator로 학습시켜 loss를 구한다!
                          # 가짜 이미지를 잘 구분할수록 loss는 줄어들고 반대의 경우 loss는 커지겠지!

        # 생성자가 만든 가짜 이미지를 구분자에 넣는다
        D_result_from_fake = D(fake_data)

        # 구분자의 출력값이 정답인 0에서 멀수록 loss가 높아진다
        D_loss_fake = criterion(D_result_from_fake, target_fake)

        ## Multi-Loss
        # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
        D_loss = D_loss_real + D_loss_fake

        # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
        D.zero_grad()

        # 역전파를 통해 매개변수의 loss에 대한 미분값을 계산한다.
        D_loss.backward()

        # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트하낟.
        D_optimizer.step()
        
# <code 8> 생성자 학습시키기
# 잠깐, 위에서 생성자 학습시키거 아녔어?? Fake Image Data 생성했는데?!
# 일단 진행해보자.

        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        z = z.cuda()
#         z = z.cuda()

        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)

        # 생성자가 만든 가짜 이미지를 구분자에 넣는다
        D_result_from_fake = D(fake_data)
        # 생성자의 입장에서 구분자의 출력값이 1에서 멀어질수록 loss가 높아진다.
        # 위조지폐범의 입장에서 경찰이 진짜 지폐로 분류하기 원할테니깐!(1은 진짜지폐로 분류!)
        G_loss = criterion(D_result_from_fake, target_real)

        # 생성자의 매개 변수의 미분값을 0로 초기화
        G.zero_grad()

        # 역전파를 통해 매개 변수의 loss에 대한 미분값 계산
        G_loss.backward()

        # 최적화 기법을 이용해 생성자의 매개 변수 업데이트
        G_optimizer.step()

    print(epoch, "update D_Loss:", D_loss)
    print(epoch, "update G_Loss:", G_loss)

# 위에서는 Discriminator의 loss를 학습하기 위해 Generator로 Fake Image를 만들어 학습했다면
# <Code 8>은 Generator를 학습시키기 위함이군!

RuntimeError: Expected object of device type cuda but got device type cpu for argument #2 'mat1' in call to _th_addmm